# Instruction Presentation

In this section we will learn how to present instructions to particiapnts in bloks. For this purpose, we will reference instructions that we have stored an a dictinary named `instrucitons` (see previous section). This section is organized as follows:
- Problem: Text Presentation in PyGame
- Solution: Blocked Text Presentation
- Functional Text Presentation Example
---

## Problem: Text Presentation in PyGame

Presenting text that spans multiple lines is not a trivail task in PyGame. That is why we cover it so late in the course. But what about the text presentation that we have already covered? In previous sections we have already seen how individual items can be presented one at a time. What if we simply use thus method of text presentation with a longer text? To see what would happen, consider the following example in which a multiline text is loaded and presented using the PyGame text rendering method:
```python
# import pygame modules
import pygame, os

# initialize pygame
pygame.init()

# create screen
screen = pygame.display.set_mode((500, 500))
pygame.display.set_caption("Problem Rendering Multiline Text")

# create a font object
# parameters are 'system font type' and 'size'
font = pygame.font.SysFont("Arial", 35)

# === loading instructions text === #
# get path
absPath = os.path.abspath(os.curdir)
instPath = os.path.join(absPath, "instructions/")

# defining instructions loading function
def load_instructions(filename):
    """loads instructions from a text file"""

    with open(instPath + filename, 'r') as file:
        infile = file.read()

    return infile

# ===================================================== #

# === section that creates the rendered text object === #
# create a text string, text color, background color, and a rendered text object
# params of the render method are 'text', anti-aliasing, color and background color
text = load_instructions("welcome.txt")
textColor = (0, 0, 0) # text is black
bgColor = (160, 160, 160) # bgColor will be light grey
textItem = font.render(text, True, textColor, bgColor)

# specify positions
# first we get the textItem rectangle and the rectangle of the screen
textRect = textItem.get_rect()
screenRect = screen.get_rect()

# next, we place the rect at the center of the screen
textRect.center = screenRect.center
# ===================================================== #

# change color of screen and draw everything
screen.fill(bgColor)
# blit text with the textRect as positional argument
screen.blit(textItem, textRect)
# flip to foreground
pygame.display.flip()

# wait for 8 seconds
pygame.time.wait(8000)

```

This code produces the follwing result:<br>
<img src="ProblemTextrendering.png" alt="ProblemTextrendering" align="left">

We see that the `render` function in PyGame does not handle multiline strings well. In fact, the entire text is rednered in one line, without considering the newline breakpoints in the text.

## Solution: Blocked Text Presentation

In order to solve this problem, we need to write a custom function that takes care of rendering multiline text to the screen. We have implemented such a function inside a module that is located here:<br><br>
**[TextPresenter.py](https://github.com/imarevic/PsyPythonCourse/blob/master/notebooks/Chapter8/TextPresenter.py)**

If we save this module in the same directory than our python script, we can use it as follows:
```python
# import pygame modules
import pygame, os
import TextPresenter

# initialize pygame
pygame.init()

# create screen
screen = pygame.display.set_mode((700, 700))
pygame.display.set_caption("Solution Rendering Multiline Text")

# create a font object
# parameters are 'system font type' and 'size'
font = pygame.font.SysFont("Arial", 28)

# === loading instructions text === #
# get path
absPath = os.path.abspath(os.curdir)
instPath = os.path.join(absPath, "instructions/")

# defining instructions loading function
def load_instructions(filename):
    """loads instructions from a text file"""

    with open(instPath + filename, 'r') as file:
        infile = file.read()

    return infile

# ===================================================== #

# === section that creates the rendered text object === #
# create a text string, text color, background color, and a rendered text object
# params of the render method are 'text', anti-aliasing, color and background color
text = load_instructions("welcome.txt")
textColor = (0, 0, 0) # text is black
bgColor = (160, 160, 160) # bgColor will be light grey
screenRect = screen.get_rect() # get screen rect

# define width and height of text
textwidth = screenRect.width - (screenRect.width//10)
textheight = screenRect.height - (screenRect.height//10)

# create instruction block object
instWelcome = TextPresenter.text_object(text, font, textwidth, textheight)

# ===================================================== #

# change color of screen and draw everything
screen.fill(bgColor)
# blit text ate center of screen
screen.blit(instWelcome, (screenRect.centerx - (textwidth // 2),
                          screenRect.centery - (textheight // 2)))
# flip to foreground
pygame.display.flip()

# wait for 8 seconds
pygame.time.wait(8000)

```

Executing this code produces the following result:<br>
<img src="SolutionTextrendering.png" alt="SolutionTextrendering" align="left">

**Note the important parts of the above script:**<br><br>
**1)** We import the module using `import TextPresenter` at the beginning of the program.<br><br>
**2)** We define the width and height of the textblock, so it does not span the entire screen:
```python
# define width and height of text
textwidth = screenRect.width - (screenRect.width//10)
textheight = screenRect.height - (screenRect.height//10)
```
**3)** We create the textblock object by calling the `text_object` funtion from the `TextPresenter` module:
```python
# create instruction block object
instWelcome = TextPresenter.text_object(text, font, textwidth, textheight)
```
The parameters are **text**, **font**, **width**, **height** !
<br><br>
**4)** Finally we blit the center of the textblock object onto the center of our screen surface:
```python
# blit text at center of screen
screen.blit(instWelcome, (screenRect.centerx - (textwidth // 2),
                          screenRect.centery - (textheight // 2)))
```

## Functional Text Presentation Example

The follwong code illustrates a functional way of presenting blocked text to participants. In this example we will present **3 instruction blocks** to participants and allow for advancing to the next textblock via pressing **RETURN**. For the final texblock, participants can continue via pressing **SPACE**. Thus, this program also illustrates jow different event types can be procesed:<br>
Here is the code:
```python
# import pygame modules
import pygame, os, sys
import TextPresenter

# === define global program parameters in a dict === #
expGlobals = {"bgColor" : (180, 180, 180), # bg is light grey
              "textColor" : (0, 0, 0), # text is black
              "screenSize" : (800, 600), # set screen screenSize
              "FPS" : 60, # frames per second
              "screen" : None, # placeholder for screen instance
              "screenRect" : None, # placeholder for screen rectangle
              "font" : None, # placeholder for font
              "instWidth" : None, # placeholder for instruction width
              "instHeight" : None, # placeholder for instruction height
              "absPath" : None, # placeholder for absolute path
              "instPath" : None, # placeholdr for relative path
              "continue" : 0 # boolean value to control continue events
              }

instructions = {"welcome" : None, # placeholder for welcome text
                "studyPurpose1" : None, # placeholder for purpose 1 text
                "studyPurpose2" : None # placeholder for purpose 2 text
                }


def run_experiment():
    """runs the experiment."""

    # initialize pygame and font
    init_pygame(expGlobals["screenSize"], expGlobals["FPS"])

    # start welcome block
    start_welcome_block()

    # exit Experiment
    quit_pygame()



def init_pygame(screenSize, FPS):
    """
    initializes pygame backends explicitly with
    predefined settings.
    """

    # initialize pygame modules
    pygame.init()

    # get nstructions path
    expGlobals["absPath"] = os.path.abspath(os.curdir)
    expGlobals["instPath"] = os.path.join(expGlobals["absPath"], "instructions/")

    # load all instructions
    instructions["welcome"] = load_instructions("welcome.txt")
    instructions["studyPurpose1"] = load_instructions("studyintro1.txt")
    instructions["studyPurpose2"] = load_instructions("studyintro2.txt")

    # define screen Settings
    expGlobals["screen"] = pygame.display.set_mode(expGlobals["screenSize"])
    pygame.display.set_caption("Learning Experiment")

    # set frame rate
    clock = pygame.time.Clock()
    clock.tick(expGlobals["FPS"])

    # get screen rect and set font
    expGlobals["font"] = pygame.font.SysFont("Arial", 40)
    expGlobals["screenRect"] = expGlobals["screen"].get_rect()

    # set instruciton text width and height
    expGlobals["instWidth"] = expGlobals["screenSize"][0] - (expGlobals["screenSize"][0] // 10)
    expGlobals["instHeight"] = expGlobals["screenSize"][1] - (expGlobals["screenSize"][1] // 10)

def load_instructions(filename):
    """loads instructions from a text file"""

    with open(expGlobals["instPath"] + filename, 'r') as file:
        infile = file.read()

    return infile

def process_continue_event():
    """processes continue events."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_RETURN:
                expGlobals["continue"] = 1

def process_space_event():
    """processes continue events."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_SPACE:
                expGlobals["continue"] = 1

def start_welcome_block():
    """presents welcome instructions to participant."""

    # set background
    expGlobals["screen"].fill(expGlobals["bgColor"])
    expGlobals["continue"] = 0

    while expGlobals["continue"] != 1:

        # create welcome instruction object
        welcomeInst = TextPresenter.text_object(instructions["welcome"], expGlobals["font"],
                                                    expGlobals["instWidth"], expGlobals["instHeight"])
        # blit instructions to screen
        expGlobals["screen"].blit(welcomeInst, (expGlobals["screenRect"].centerx - (expGlobals["instWidth"] // 2),
                                               expGlobals["screenRect"].centery - (expGlobals["instHeight"] // 2)))
        # flip to foreground
        pygame.display.flip()

        # process continue event
        process_continue_event()

    # redrawing background and reseting continue to 0
    expGlobals["screen"].fill(expGlobals["bgColor"])
    expGlobals["continue"] = 0
    while expGlobals["continue"] != 1:

        # create welcome instruction object
        studyInst1 = TextPresenter.text_object(instructions["studyPurpose1"], expGlobals["font"],
                                                    expGlobals["instWidth"], expGlobals["instHeight"])
        # blit instructions to screen
        expGlobals["screen"].blit(studyInst1, (expGlobals["screenRect"].centerx - (expGlobals["instWidth"] // 2),
                                               expGlobals["screenRect"].centery - (expGlobals["instHeight"] // 2)))
        # flip to foreground
        pygame.display.flip()

        # process continue event
        process_continue_event()

    # redrawing background and reseting continue to 0
    expGlobals["screen"].fill(expGlobals["bgColor"])
    expGlobals["continue"] = 0
    while expGlobals["continue"] != 1:

        # create welcome instruction object
        studyInst2 = TextPresenter.text_object(instructions["studyPurpose2"], expGlobals["font"],
                                                    expGlobals["instWidth"], expGlobals["instHeight"])
        # blit instructions to screen
        expGlobals["screen"].blit(studyInst2, (expGlobals["screenRect"].centerx - (expGlobals["instWidth"] // 2),
                                               expGlobals["screenRect"].centery - (expGlobals["instHeight"] // 2)))
        # flip to foreground
        pygame.display.flip()

        # process continue event
        process_space_event()


def quit_pygame():
    """exits pygame explicitly."""
    # quit program
    pygame.quit()


# == start the program == #
if __name__ == '__main__':
    run_experiment()

```

Note the follwing about the above program:<br>
**1)** All instructions are loaded and stored in a dictionary named `instructions`. We fill that dictionary when we call the function `init_pygame()`.<br><br>
**2)** We define **two different** functions for handling continue events. One enables pressing *RETURN* and the other *SPACE*:
```python
def process_continue_event():
    """processes continue events."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_RETURN:
                expGlobals["continue"] = 1

def process_space_event():
    """processes continue events."""
    for event in pygame.event.get():
        # handle quit event
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        elif event.type == pygame.KEYDOWN:
        # respond to a keypress
            if event.key == pygame.K_SPACE:
                expGlobals["continue"] = 1
```
**3)** We do all instruction presentation inside a while loop. For this purpose we define a boolean variable `expGlobals["continue"]`. The instruction presentation is done 3 times in sequence within one function named `start_welcome_block()`. Please note that we could have broken this down into smaller functions as well, with each function presenting one text block. For convenience, however, in this example all the instructions are presented within one function. Here we need to reset `expGlobals["continue"]` to `0` everytime we start a new `while-loop`.